# Importing data

## Importing Twitter Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

file_path = '/content/drive/My Drive/APS360 Group/Data preprocessing/Text data/Processed_tweets.csv'
df = pd.read_csv(file_path)
df

,clean_text,category,processed_tweets
0,when modi promised “minimum government maximum...,-1.0,"when modi promise "" minimum government maximum..."
1,talk all the nonsense and continue all the dra...,0.0,talk all the nonsense and continue all the dra...
2,what did just say vote for modi welcome bjp t...,1.0,what do just say vote for modi welcome bjp tel...
3,asking his supporters prefix chowkidar their n...,1.0,ask his supporter prefix chowkidar their name ...
4,answer who among these the most powerful world...,1.0,answer who among these the most powerful world...
...,...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0,why these 456 crore pay neerav modi not recove...
162976,dear rss terrorist payal gawar what about modi...,-1.0,dear rss terrorist payal gawar what about modi...
162977,did you cover her interaction forum where she ...,0.0,do you cover her interaction forum where she l...
162978,there big project came into india modi dream p...,0.0,there big project come into india modi dream p...


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

df_processed = df[['category', 'processed_tweets']]
df_processed = df_processed.rename(columns={'category': 'labels'})

# Limit to the first 50000 entries
df_processed = df_processed.iloc[:50000]
df_processed = df_processed.reset_index(drop=True)

# Separate df_processed by class
df_negative = df_processed[df_processed['labels'] == -1.0]
df_neutral = df_processed[df_processed['labels'] == 0.0]
df_positive = df_processed[df_processed['labels'] == 1.0]

# Find the number of samples in the minority class
min_samples = min(len(df_negative), len(df_neutral), len(df_positive))
print("min_samples ", min_samples)

# Resample other classes to match the minority class size
df_negative_balanced = df_negative.sample(n=min_samples, random_state=42)
df_neutral_balanced = df_neutral.sample(n=min_samples, random_state=42)
df_positive_balanced = df_positive.sample(n=min_samples, random_state=42)

# Concatenate the balanced datasets
df_balanced = pd.concat([df_negative_balanced, df_neutral_balanced, df_positive_balanced])

# Shuffle the balanced dataset
df_processed = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

#ADDEDDDDDDDDDD
# After df_processed is balanced and before the split
df_processed['labels'] = df_processed['labels'].map({-1: 0, 0: 1, 1: 2})


# Now, proceed with splitting the balanced df_processed into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(df_processed['processed_tweets'], df_processed['labels'], test_size=0.25, random_state=42)

test_val_proportion = 0.15 / 0.25
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=test_val_proportion, random_state=42)

# Convert to lists for compatibility with further processing
X_train = X_train.tolist()
y_train = y_train.tolist()
X_val = X_val.tolist()
y_val = y_val.tolist()
X_test = X_test.tolist()
y_test = y_test.tolist()

min_samples  11043


## Importing Rotten Tomatoes Data

In [5]:
file_path = '/content/drive/My Drive/APS360 Group/Data preprocessing/Text data/Processed_RT.csv'
df_rt = pd.read_csv(file_path)
df_rt

,phrase,sentiment_label
0,! Alas,0
1,! Brilliant,1
2,! Brilliant !,1
3,! Brilliant ! ',1
4,! Gollum 's ` performance ' is incredible,1
...,...,...
224899,zone,0
224900,zoning,0
224901,zoning ordinances,0
224902,zoning ordinances to protect your community fr...,-1


# Coding the model

install transformer library

In [6]:
!pip install transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.8 MB/s eta 0:00:00


Tokenization

In [ ]:
def tokenize_data(texts):
    # Filter out NaN values and convert to strings
    texts = [str(text) for text in texts if pd.notnull(text)]
    return tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

train_encodings = tokenize_data(X_train)
val_encodings = tokenize_data(X_val)
test_encodings = tokenize_data(X_test)

Convert to PyTorch Datasets

In [ ]:
!pip install torch

# Import the necessary PyTorch module for TensorDataset and DataLoader
import torch
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
print(f"Number of training examples: {len(train_encodings['input_ids'])}")
print(f"Number of training labels: {len(y_train)}")

Number of training examples: 24845
Number of training labels: 24845


In [ ]:
if len(train_encodings['input_ids']) != len(y_train):
    y_train.pop()  # Removes the last element; adjust the index as needed

# Verify the lengths again
assert len(train_encodings['input_ids']) == len(y_train), "The lengths should now be equal."

Create TensorDataset Objects

In [ ]:
train_dataset = TensorDataset(
    train_encodings['input_ids'],
    train_encodings['attention_mask'],
    torch.tensor(y_train)
)

val_dataset = TensorDataset(
    val_encodings['input_ids'],
    val_encodings['attention_mask'],
    torch.tensor(y_val)
)

test_dataset = TensorDataset(
    test_encodings['input_ids'],
    test_encodings['attention_mask'],
    torch.tensor(y_test)
)

Create DataLoaders

In [ ]:
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Define the DistilBERT Model for Sequence Classification

In [ ]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=3
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Set Up Training

In [ ]:
!pip install transformers[torch]
!pip install accelerate

In [ ]:
!pip show accelerate

Name: accelerate
Version: 0.28.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory for model checkpoints
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=32,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Weight decay for optimizer
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`